In [1]:
"""Example usage:
  python object_detection/dataset_tools/create_oid_tf_record.py \
    --input_annotations_csv=/path/to/input/annotations-human-bbox.csv \
    --input_images_directory=/path/to/input/image_pixels_directory \
    --input_label_map=/path/to/input/labels_bbox_545.labelmap \
    --output_tf_record_path_prefix=/path/to/output/prefix.tfrecord
CSVs with bounding box annotations and image metadata (including the image URLs)
can be downloaded from the Open Images GitHub repository:
https://github.com/openimages/dataset
This script will include every image found in the input_images_directory in the
output TFRecord, even if the image has no corresponding bounding box annotations
in the input_annotations_csv.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import contextlib2
import pandas as pd
import tensorflow as tf

from object_detection.dataset_tools import oid_tfrecord_creation
from object_detection.utils import label_map_util

## v4, eval dataset

In [2]:
input_annotations_csv='/root/2018_04/validation/annotations-human-bbox.csv'
input_images_directory='/root/raw_images_validation'
input_label_map='../object_detection/data/oid_v4_label_map.pbtxt'
# !mkdir test_tfrecords
# output_tf_record_path_prefix='test_tfrecords/test.tfrecord'
num_shards=100

In [5]:
tf.logging.set_verbosity(tf.logging.INFO)


all_annotations_v4_eval = pd.read_csv(input_annotations_csv)
all_images = tf.gfile.Glob(
  os.path.join(input_images_directory, '*.jpg'))
all_image_ids = [os.path.splitext(os.path.basename(v))[0] for v in all_images]
all_image_ids = pd.DataFrame({'ImageID': all_image_ids})
# all_annotations = pd.concat([all_annotations, all_image_ids])

tf.logging.log(tf.logging.INFO, 'Found %d images...', len(all_image_ids))

INFO:tensorflow:Found 41620 images...


In [4]:
label_map_600 = label_map_util.get_label_map_dict(input_label_map)
label_map_600_info = label_map_util.load_labelmap(input_label_map)

In [6]:
all_annotations_v4_eval.groupby("LabelName").size()

LabelName
/m/011k07       53
/m/012074       22
/m/0120dh       36
/m/01226z       64
/m/012n7d       13
/m/012w5l       20
/m/012xff       16
/m/0130jx       54
/m/0138tl      437
/m/013y1f       21
/m/01432t        9
/m/014j1m      102
/m/014sv8     4304
/m/014trl       50
/m/014y4n      253
/m/0152hh        4
/m/01599       118
/m/015h_t      153
/m/015p6       943
/m/015qbp        2
/m/015qff       49
/m/015wgc       40
/m/015x4r      100
/m/015x5n       21
/m/0162_1        7
/m/0167gd       77
/m/016m2d       24
/m/0174k2       39
/m/0174n1       54
/m/0176mf       13
              ... 
/m/0k1tl        49
/m/0k4j       9381
/m/0k5j        186
/m/0k65p      4123
/m/0km7z         3
/m/0kmg4        30
/m/0kpqd        32
/m/0kpt_        16
/m/0ky7b         2
/m/0l14j_       14
/m/0l3ms         9
/m/0l515        90
/m/0ll1f78     116
/m/0llzx        25
/m/0lt4_         6
/m/0m53l        26
/m/0mcx2        29
/m/0mkg         24
/m/0mw_6         2
/m/0n28_        66
/m/0nl46        19
/m

In [7]:
input_label_map='../object_detection/data/oid_bbox_trainable_label_map_add2.pbtxt'
label_map_547 = label_map_util.get_label_map_dict(input_label_map)
label_map_547_info = label_map_util.load_labelmap(input_label_map)

In [12]:
label_map_547

{u'/m/011k07': 241,
 u'/m/012074': 510,
 u'/m/0120dh': 309,
 u'/m/01226z': 118,
 u'/m/012n7d': 436,
 u'/m/012w5l': 321,
 u'/m/012xff': 540,
 u'/m/012ysf': 527,
 u'/m/0130jx': 231,
 u'/m/0138tl': 48,
 u'/m/013y1f': 421,
 u'/m/014j1m': 265,
 u'/m/014sv8': 62,
 u'/m/014trl': 246,
 u'/m/014y4n': 164,
 u'/m/0152hh': 361,
 u'/m/01599': 84,
 u'/m/015h_t': 184,
 u'/m/015p6': 27,
 u'/m/015qbp': 546,
 u'/m/015qff': 190,
 u'/m/015wgc': 482,
 u'/m/015x4r': 390,
 u'/m/015x5n': 476,
 u'/m/0162_1': 497,
 u'/m/0167gd': 138,
 u'/m/016m2d': 204,
 u'/m/0174k2': 423,
 u'/m/0174n1': 219,
 u'/m/0176mf': 440,
 u'/m/017ftj': 60,
 u'/m/018j2': 456,
 u'/m/018p4k': 182,
 u'/m/018xm': 89,
 u'/m/01940j': 322,
 u'/m/0199g': 43,
 u'/m/019dx1': 155,
 u'/m/019h78': 481,
 u'/m/019jd': 36,
 u'/m/019w40': 176,
 u'/m/01_5g': 387,
 u'/m/01_bhs': 83,
 u'/m/01b638': 212,
 u'/m/01b7fy': 275,
 u'/m/01b9xk': 420,
 u'/m/01bfm9': 95,
 u'/m/01bjv': 87,
 u'/m/01bl7v': 21,
 u'/m/01bqk0': 59,
 u'/m/01btn': 406,
 u'/m/01c648': 105,
 u

In [13]:
eval_dict = all_annotations_v4_eval.groupby("LabelName").size().to_dict()

In [14]:
notInEval = []
for label in label_map_547.keys():
    if label not in eval_dict:
        notInEval.append(label)
print(notInEval)

[u'/m/0cvnqh', u'/m/027rl48', u'/m/0420v5', u'/m/012ysf']


In [15]:
## v4, test dataset

In [16]:
input_annotations_csv='/root/2018_04/test/annotations-human-bbox.csv'
input_images_directory='/root/raw_images_test'
input_label_map='../object_detection/data/oid_v4_label_map.pbtxt'
# !mkdir test_tfrecords
# output_tf_record_path_prefix='test_tfrecords/test.tfrecord'
num_shards=100

In [17]:
tf.logging.set_verbosity(tf.logging.INFO)


all_annotations_v4_test = pd.read_csv(input_annotations_csv)
all_images = tf.gfile.Glob(
  os.path.join(input_images_directory, '*.jpg'))
all_image_ids = [os.path.splitext(os.path.basename(v))[0] for v in all_images]
all_image_ids = pd.DataFrame({'ImageID': all_image_ids})
# all_annotations = pd.concat([all_annotations, all_image_ids])

tf.logging.log(tf.logging.INFO, 'Found %d images...', len(all_image_ids))

INFO:tensorflow:Found 125436 images...


In [18]:
len(all_annotations_v4_test.groupby("LabelName").size())

576

In [19]:
test_dict = all_annotations_v4_test.groupby("LabelName").size().to_dict()
notInEval = []
for label in label_map_547.keys():
    if label not in test_dict:
        notInEval.append(label)
print(notInEval)

[u'/m/0cvnqh', u'/m/027rl48']


In [ ]:
## 500 label


In [ ]:
input_label_map='../object_detection/data/oid_bbox_trainable_label_map_add2.pbtxt'
label_map_547 = label_map_util.get_label_map_dict(input_label_map)
label_map_547_info = label_map_util.load_labelmap(input_label_map)